In [1]:
import pandas as pd
import numpy as np
import gurobipy as gp
from gurobipy import GRB

# Load data
travel_times = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/travel_times.csv')
ward_data = pd.read_csv('/Users/jay/Desktop/DISS/wards_data.csv')
fire_stations = pd.read_csv('/Users/jay/Desktop/DISS/DISS_Data/stations.csv')
fire_stations['station_id'] = fire_stations.index

# Define demand locations and potential fire station sites
demand_locations = ward_data['ward_id'].tolist()
potential_fire_station_sites = fire_stations['station_id'].tolist()

# Define demand weights
demand_weights = ward_data.set_index('ward_id')['demand_weight'].to_dict()

# Create a travel time matrix
travel_time_matrix = travel_times.pivot(index='ward_id', columns='station_id', values='travel_time').fillna(np.inf)

# Ensure all potential fire station sites are in the travel time matrix
travel_time_matrix = travel_time_matrix.loc[demand_locations, potential_fire_station_sites]

# Print the travel time matrix to verify its correctness
print("Travel Time Matrix:")
print(travel_time_matrix)

# Create a new model
model = gp.Model('MCLP')

# Decision variables
x = model.addVars(demand_locations, vtype=GRB.BINARY, name='X')  # Whether a ward is covered
y = model.addVars(potential_fire_station_sites, vtype=GRB.BINARY, name='Y')  # Whether a station is sited

# Objective: Maximize the total weighted demand covered by selected fire stations
objective = gp.quicksum(demand_weights[ward] * x[ward] for ward in demand_locations)
model.setObjective(objective, GRB.MAXIMIZE)

# Constraints
response_time_standard = 360  # Response time standard in seconds (6 minutes)

# Coverage constraint: Ensure that if a demand location is covered, at least one fire station within the response time standard must be sited
for ward in demand_locations:
    model.addConstr(
        gp.quicksum(y[station] for station in potential_fire_station_sites if travel_time_matrix.at[ward, station] <= response_time_standard) >= x[ward],
        name=f"coverage_{ward}"
    )

# Number of fire stations constraint: Specify the number of fire stations to be established
number_of_fire_stations = 102  # Adjust based on your requirement
model.addConstr(gp.quicksum(y[station] for station in potential_fire_station_sites) == number_of_fire_stations)

# Optimize the model
model.optimize()

# Print the results
if model.status == GRB.OPTIMAL:
    selected_stations = [station for station in potential_fire_station_sites if y[station].x > 0.5]
    print("Selected Fire Stations:", selected_stations)
    covered_wards = [ward for ward in demand_locations if x[ward].x > 0.5]
    print("Covered Wards:", covered_wards)
    
    # Additional analysis (optional)
    covered_demand = sum(demand_weights[ward] for ward in covered_wards)
    print(f"Total covered demand: {covered_demand}")

    # Print out the full details of each selected station and the wards they cover
    for station in selected_stations:
        covered_by_station = [ward for ward in demand_locations if travel_time_matrix.at[ward, station] <= response_time_standard]
        print(f"Fire Station {station} covers wards: {covered_by_station}")

else:
    print("No optimal solution found.")


Travel Time Matrix:
station_id   0     1     2     3     4     5     6     7     8     9    ...  \
ward_id                                                                 ...   
E05000405   4104  3567  3932  3855  4011  3768  2976  3097  3453  3406  ...   
E05000414   4550  4014  3920  3878  3784  3541  3422  3542  3899  3178  ...   
E05000401   4908  4372  4401  3851  3757  3514  3780  3901  4257  3203  ...   
E05000400   4837  4301  4324  3774  3679  3437  3710  3830  4187  3074  ...   
E05000402   4912  4594  4274  3724  3630  3387  3784  3904  4261  3025  ...   
...          ...   ...   ...   ...   ...   ...   ...   ...   ...   ...  ...   
E05000042    286   778  2708  3186  2661  3010  2609  2368  2533  3637  ...   
E05000030    269   862  2988  3467  2941  3291  2401  2161  2326  3918  ...   
E05000029    556  1115  2506  2984  2459  2808  2477  2237  2402  3436  ...   
E05000037    359   517  2643  3122  2597  2946  2356  2115  2280  3573  ...   
E05009297   2439  1781  3033  27

Fire Station 4 covers wards: ['E05000051', 'E05000049', 'E05000063', 'E05000061', 'E05000270']
Fire Station 5 covers wards: ['E05000062', 'E05000046', 'E05000051', 'E05000055']
Fire Station 6 covers wards: ['E05011227', 'E05011230']
Fire Station 7 covers wards: ['E05011223', 'E05011219', 'E05011228', 'E05011233', 'E05011217']
Fire Station 8 covers wards: ['E05011228', 'E05011218', 'E05011232', 'E05011233', 'E05011217', 'E05011225']
Fire Station 10 covers wards: ['E05000183', 'E05000104', 'E05000101']
Fire Station 11 covers wards: ['E05000092', 'E05000091', 'E05000105', 'E05000087', 'E05000089']
Fire Station 12 covers wards: ['E05000112', 'E05000120', 'E05000123', 'E05000113']
Fire Station 15 covers wards: ['E05000122']
Fire Station 16 covers wards: ['E05000142']
Fire Station 17 covers wards: ['E05000045', 'E05000132', 'E05000133', 'E05000135', 'E05000145']
Fire Station 18 covers wards: ['E05000137', 'E05000134', 'E05000139', 'E05000378', 'E05000376']
Fire Station 20 covers wards: ['E05

In [2]:
# Extract the deleted fire stations (not in selected_stations)
all_station_ids = fire_stations.index.tolist()
selected_stations = [station for station in all_station_ids if station in selected_stations]

# Filter the datasets based on the results
selected_stations_df = fire_stations.loc[selected_stations]
covered_wards_df = ward_data[ward_data['ward_id'].isin(covered_wards)]

# Save the filtered datasets
selected_stations_df.to_csv('selected_stations2.csv', index=False)
covered_wards_df.to_csv('covered_wards2.csv', index=False)


selected_stations_df.head(), covered_wards_df.head()

(        name                                            address  \
 0   Dagenham  Rainham Road North, Dagenham, Greater London R...   
 1    Barking                   Alfred's Way, Greater London, UK   
 2     Barnet   144 Station Road, Barnet, Greater London EN5, UK   
 3  Mill Hill  10 Hartley Avenue, Edgware, London, Greater Lo...   
 4   Finchley  227 Long Lane, London Borough of Barnet, Londo...   
 
                 borough   latitude  longitude  station_id  
 0  Barking and Dagenham  51.558785   0.157054           0  
 1  Barking and Dagenham  51.532104   0.103288           1  
 2                Barnet  51.646995  -0.185843           2  
 3                Barnet  51.615091  -0.243472           3  
 4                Barnet  51.597836  -0.179377           4  ,
             NAME    ward_id              DISTRICT  LAGSSCODE  HECTARES  \
 2     Berrylands  E05000401  Kingston upon Thames  E09000021   145.390   
 3      Alexandra  E05000400  Kingston upon Thames  E09000021   268.506  